load the pre-processed dataset

In [ ]:
df = pd.read_csv("df.csv")

In [ ]:
import re
import nltk
import gensim
import tiktoken
import pandas as pd
import numpy as np
import seaborn as sns
from umap import UMAP
from hdbscan import HDBSCAN
import matplotlib.pyplot as plt
import gensim.corpora as corpora
from bertopic import BERTopic
from sklearn.cluster import KMeans
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import sent_tokenize
from sklearn.metrics import silhouette_score
from sentence_transformers import SentenceTransformer
from bertopic.representation import OpenAI
import matplotlib.pyplot as plt
from bertopic.representation import TextGeneration
from sklearn.cluster import KMeans
nlp = spacy.load("en_core_web_sm")
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
import locale
print(locale.getpreferredencoding())

UTF-8


In [ ]:
from torch import cuda

# model_id = 'meta-llama/Llama-2-7b-chat-hf'
model_id = 'meta-llama/Llama-2-13b-hf'
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

print(device)

cuda:0


In [ ]:
!huggingface-cli login

In [ ]:
!pip -q install torch transformers

In [ ]:
# restart session after installing all stuffs

import os
import torch
# from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
# from peft import LoraConfig, PeftModel
# from trl import SFTTrainer


In [ ]:
from torch import bfloat16
import transformers

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit quantization
    bnb_4bit_quant_type='nf4',  # Normalized float 4
    bnb_4bit_use_double_quant=True,  # Second quantization after the first
    bnb_4bit_compute_dtype=bfloat16  # Computation type
)

# Llama 2 Tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

# Llama 2 Model
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto',
)
model.eval()

# Our text generator
generator = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    task='text-generation',
    temperature=0.1,
    max_new_tokens=500,
    repetition_penalty=1.1
)

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
import locale
print(locale.getpreferredencoding())

UTF-8


In [ ]:
# restart session after installing all stuffs

import os
import torch
# from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
# from peft import LoraConfig, PeftModel
# from trl import SFTTrainer


In [ ]:
# System prompt describes information given to all conversations
system_prompt = """
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant for labeling topics.
<</SYS>>
"""

main_prompt = """
I have a topic that contains the following documents:
[DOCUMENTS]
The topic is described by the following keywords: [KEYWORDS]

Based on the information above, extract a short topic label and related texts in the following format:
topic: <topic label> text: <texts>
"""

# prompt = system_prompt + example_prompt + main_prompt

prompt = system_prompt + main_prompt

seed_words = ["Language","Dataset","Corpus Size","Computational","Machine Translation","Tokenization","Costs"
  "Interpretability", "Morphology","Semantic","Memory","Skewed distributions","Biased Distributions","Small size",
                "Generalizability","Bias","Hyperparameter","Hardware","Real world","Robustness", "Noisy", "Time",
                "Annotations", "Evaluation", "Diversity", "Segmentation", "Metrics"]


In [ ]:
# Text generation with Llama 2
llama2 = TextGeneration(generator, prompt=prompt)
representation_model = {
    "Llama2": llama2,
}
vectorizer_model = CountVectorizer(stop_words="english")

sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(df['Text'], show_progress_bar=False)

# Create our BERTopic model
topic_model = BERTopic(representation_model=representation_model,vectorizer_model=vectorizer_model,
                        min_topic_size=10,seed_topic_list = seed_words, verbose=True,calculate_probabilities=True)

topics, probs = topic_model.fit_transform(df['Text'], embeddings)


In [ ]:
documents = pd.DataFrame({"Document": df['Text'],
                          "ID": range(len(df['Text'])),
                          "Topic": topics})
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
cleaned_docs = topic_model._preprocess_text(documents_per_topic.Document.values)

# Extract vectorizer and analyzer from BERTopic
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names_out()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in topic_model.get_topic(topic)]
               for topic in range(len(set(topics))-1)]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words,
                                 texts=tokens,
                                 corpus=corpus,
                                 dictionary=dictionary,
                                 coherence='c_v')
coherence = coherence_model.get_coherence()

In [ ]:
print("coherence score is",coherence)

coherence score is 0.5169884630792903


In [ ]:
import numpy as np
from sklearn.metrics import silhouette_score

umap_embeddings = topic_model.umap_model.transform(embeddings)
indices = [index for index, topic in enumerate(topics) if topic != -1]
X = umap_embeddings[np.array(indices)]
labels = [topic for index, topic in enumerate(topics) if topic != -1]

# Calculate silhouette score
silhouette_score(X, labels)

0.57781935

In [ ]:
most_representative_words_llama = topic_model.get_topic_info()
most_representative_words_llama

,Topic,Count,Name,Representation,Llama2,Representative_Docs
0,-1,1007,-1_models_model_data_work,"[models, model, data, work, language, training...",[\n[EXT] <<SYS>>\n\ntopic: descriptive masked ...,[Model sizes and comparability as we have poin...
1,0,260,0_languages_language_translation_data,"[languages, language, translation, data, model...",[\n[OUT] >>SYS<<\nTopic: I have a topic that c...,[Limitation indicates that the transformer mod...
2,1,97,1_dialogue_evaluation_chatgpt_conversations,"[dialogue, evaluation, chatgpt, conversations,...",[\n[OUT] <<SYS>>\ntopic: dialogue evaluation c...,[The current dialogue system is mainly based o...
3,2,89,2_visual_image_multimodal_images,"[visual, image, multimodal, images, model, wor...",[\n[OUT] >>SYS<<\nTopic: Limitations on the ev...,[Despite the significant advancements made by ...
4,3,65,3_reasoning_gpt3_models_prompting,"[reasoning, gpt3, models, prompting, language,...",[\n[OUT] >>SYS<<\nTopic: reasoning\nText: gpt3...,[Our work focuses on building a two-stage fram...
5,4,54,4_summarization_summaries_summary_model,"[summarization, summaries, summary, model, lon...",[\n[OUT] >>SYS<<\nTopic: summarization\nText: ...,[We propose a set prediction network for the e...
6,5,51,5_knowledge_graph_graphs_entities,"[knowledge, graph, graphs, entities, relations...",[\n[OUT] >>SYS<<\ntopic: knowledge graph entit...,[While our approach effectively predicts the r...
7,6,48,6_clinical_medical_patients_notes,"[clinical, medical, patients, notes, patient, ...",[\n[OUT] >>SYS<<\nTopic: clinical medical pati...,[This study is based on a single clinical coho...
8,7,47,7_adversarial_attacks_attack_backdoor,"[adversarial, attacks, attack, backdoor, robus...",[\n[OUT] >>SYS<<\nTopic: adversarial attack\nT...,[Auxiliary attack and supports: votetrans with...
9,8,46,8_label_noise_entity_data,"[label, noise, entity, data, labels, weak, lab...","[\n[OUT] >>SYS<<\nTopic: label, noise, entity,...",[The limitations of this work are: in this wor...


In [ ]:
most_representative_words_llama.to_csv('gdrive/My Drive/limitations_dataset/bertopic_fine_tuning/df_all_papers_bertopic_llama_13b_zero_shots_topic_with_text_no_cos_sim.csv', index=False)  # Set index=False if you don't want to save the row indices


In [ ]:
for i in range(len(most_representative_words_llama)):
  print(most_representative_words_llama["Representation"][i])

['models', 'model', 'data', 'work', 'language', 'training', 'performance', 'tasks', 'method', 'limitation']
['languages', 'language', 'translation', 'data', 'models', 'work', 'multilingual', 'model', 'training', 'used']
['dialogue', 'evaluation', 'chatgpt', 'conversations', 'human', 'responses', 'dialogues', 'model', 'response', 'generation']
['visual', 'image', 'multimodal', 'images', 'model', 'work', 'scene', 'proposed', 'models', 'pretrained']
['reasoning', 'gpt3', 'models', 'prompting', 'language', 'task', 'explanations', 'model', 'results', 'steps']
['summarization', 'summaries', 'summary', 'model', 'long', 'document', 'datasets', 'performance', 'dialogue', 'text']
['knowledge', 'graph', 'graphs', 'entities', 'relations', 'kg', 'extraction', 'relation', 'model', 'information']
['clinical', 'medical', 'patients', 'notes', 'patient', 'data', 'report', 'evidence', 'models', 'radiology']
['adversarial', 'attacks', 'attack', 'backdoor', 'robustness', 'text', 'methods', 'textual', 'nlp'

In [ ]:
for i in range(len(most_representative_words_llama)):
  print(most_representative_words_llama["Llama2"][i])

['\n[EXT] <<SYS>>\n\ntopic: descriptive masked language modeling\ntext:\n- We propose Descriptive Masked Language Modeling (dmlm), a novel pre-training task that combines masked language modeling with word sense disambiguation.Dmlm trains a language model to predict the correct sense of a word given a context and a masked word.We show that dmlm improves the performance of language models on various downstream tasks, including sentiment analysis, question answering, and text classification.We also demonstrate that dmlm can be used to improve the performance of existing pre-trained language models, such as BERT and RoBERTa.Finally, we show that dmlm can be used to improve the performance of language models on out-of-domain data.\n- Our results suggest that dmlm is a promising new pre-training task that can improve the performance of language models on a variety of downstream tasks.\n- We propose Descriptive Masked Language Modeling (dmlm), a novel pre-training task that combines masked l

In [ ]:

for i in range(len(most_representative_words_llama)):
  print(most_representative_words_llama["Representative_Docs"][i])

['Model sizes and comparability as we have pointed out in the paper, due to computational and time constraints on the hardware we had at our disposal, we found it was unfeasible to train larger architectures.Nevertheless, we believe our comparisons between dmlm and its direct competitor, mlm, have been fair, as we have done our best to set a level playing field between the two.Thus, while we understand that this is a significant limitation in terms of comparability to larger models, we still think the results we have obtained could pave the way for further exploration in this direction.Moreover, we have performed architecture scaling experiments to show that it is important to continue research in this direction, and test dmlm’s capabilities on larger networks, while we did not perform a similar comparison with mlm because several works have already explored how mlm scales with network size.Applying dmlm only half of the time although we acknowledge that our choice to apply dmlm to onl

# visualization

In [ ]:
topic_model.visualize_topics()


In [ ]:
topic_model.visualize_heatmap()


In [ ]:
# Run the visualization with the original embeddings
topic_model.visualize_documents(df['Text'], embeddings=embeddings)

# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
topic_model.visualize_documents(df['Text'], reduced_embeddings=reduced_embeddings)

In [ ]:
from bertopic import BERTopic

# topic_model = BERTopic()
# topics, _ = topic_model.fit_transform(docs)
topic_distr, _ = topic_model.approximate_distribution(df['Text'], min_similarity=0)


100%|██████████| 3/3 [00:02<00:00,  1.14it/s]


In [ ]:
# To visualize the probabilities of topic assignment
topic_model.visualize_distribution(probs[0])

# To visualize the topic distributions in a document
topic_model.visualize_distribution(topic_distr[0])


In [ ]:
topic_model.visualize_barchart()


In [ ]:
# Calculate the topic distributions on a token-level
topic_distr, topic_token_distr = topic_model.approximate_distribution(df['Text'], calculate_tokens=True)

# Visualize the token-level distributions
df = topic_model.visualize_approximate_distribution(df['Text'][1], topic_token_distr[1])
df


100%|██████████| 3/3 [00:04<00:00,  1.43s/it]


,This,paper,argues,that,the,proposed,task,of,ellipsisdependent,reasoning,is,difficult,challenge,for,gpt,models,which,are,among,the,most,powerful,current,language,models,The,data,constructed,here,is,restricted,to,english,and,furthermore,is,restricted,to,single,form,of,ellipsis,namely,verb,phrase,ellipsis,It,may,well,be,that,other,forms,of,ellipsis,may,give,rise,to,different,effects,and,it,is,also,important,to,test,the,claims,made,here,on,other,languages
0_languages_language_translation_data,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.126,0.126,0.126,0.126,0.000,0.000,0.104,0.271,0.541,0.541,0.438,0.270,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.310,0.310,0.310,0.310
1_dialogue_evaluation_chatgpt_conversations,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.108,0.108,0.108,0.108,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2_visual_image_multimodal_images,0.000,0.000,0.000,0.125,0.249,0.249,0.249,0.125,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.101,0.241,0.241,0.241,0.140,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3_reasoning_gpt3_models_prompting,0.000,0.000,0.000,0.110,0.220,0.220,0.407,0.462,0.516,0.765,0.577,0.412,0.248,0.000,0.000,0.157,0.157,0.157,0.157,0.000,0.000,0.125,0.277,0.461,0.461,0.336,0.184,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4_summarization_summaries_summary_model,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.104,0.104,0.104,0.104,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
6_clinical_medical_patients_notes,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.147,0.147,0.147,0.147,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
7_adversarial_attacks_attack_backdoor,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.109,0.109,0.109,0.109,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
8_label_noise_entity_data,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00

In [ ]:
topic_model.visualize_hierarchy()


In [ ]:
hierarchical_topics = topic_model.hierarchical_topics(df['Text'])

topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)
# text based topic tree
tree = topic_model.get_topic_tree(hierarchical_topics)
print(tree)